In [2]:
using Pkg

In [3]:
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

# Enable printing of 1000 columns:
ENV["COLUMNS"] = 1000

   Updating registry at `C:\Users\aniru\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`


1000

In [4]:
# To Read File
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [5]:
# Summary Of Data
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [6]:
# Checking The Name Of The Columns In Data.
names(df)

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [7]:
# Checking The Number Of Responses In The Response Variable 
countmap(df.Exited)

Dict{Int64,Int64} with 2 entries:
  0 => 7963
  1 => 2037

# One Hot Encoding

In [8]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df, Not([:RowNumber, :CustomerId, :Surname, :Geography, :Gender, :Male]))

first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


# TRAIN TEST SPLIT

In [9]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75);

# MODEL BUILDING

In [10]:
fm = @formula(Exited ~ CreditScore + Age + Tenure +Balance +NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France +Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                        Coef.   Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.94972      0.168273     -11.59    <1e-30  -2.27953      -1.61991
CreditScore      -0.000406052  0.000186152   -2.18    0.0292  -0.000770904  -4.12013e-5
Age               0.0428273    0.00169399    25.28    <1e-99   0.0395071     0.0461475
Tenure           -0.00912915   0.00619733    -1.47    0.1407  -0.0212757     0.00301739
Balance           1.530

# MODEL PREDICTION AND EVALUATION

In [11]:
#  Predict The Targer Variable On Test Data
prediction = predict(logit, test)

2539-element Array{Union{Missing, Float64},1}:
 0.1312452015758905
 0.24588207372209986
 0.05252687901503972
 0.10596813449437516
 0.22947524795828766
 0.022719161681877095
 0.12695104265881046
 0.32930409020854434
 0.05758392287867415
 0.45344090672002185
 0.0487061694273981
 0.031477753569456285
 0.1572659378077667
 ⋮
 0.46310204679926387
 0.0571076667313338
 0.1426344491814511
 0.1924449518731703
 0.1589075799726788
 0.5031945430757749
 0.13700639155567873
 0.5846654806504981
 0.17385707310327475
 0.10115017127509107
 0.03691090318862209
 0.06941476900089348

In [13]:
# Convert Probability Score To Class
prediction_class = [if x<0.5 0 else 1 end for x in prediction]
prediction_df = DataFrame(y_actual = test.Exited, y_predicted = prediction_class, prob_predicted = prediction)
prediction_df.correctly_classified = prediction_df.y_actual .== prediction_df.y_predicted

2539-element BitArray{1}:
 0
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
 ⋮
 1
 1
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1

# Accuracy

In [14]:
accuracy = mean(prediction_df.correctly_classified)

0.8101614808979913

# Confusion Matrix

In [15]:
confusion_matrix = MLBase.roc(prediction_df.y_actual, prediction_df.y_predicted)

ROCNums{Int64}
  p = 532
  n = 2007
  tp = 115
  tn = 1942
  fp = 65
  fn = 417
